In [15]:
!pip install Faker
!pip install pandas 
!pip install tweepy

In [16]:
import time

from confluent_kafka import Producer
import json
import tweepy

producer = Producer(**{"bootstrap.servers": "kafka2:19092"})

In [17]:
consumer_key = "hK08N3zIwkJLrF6BVK52b3aS8"
consumer_secret = "8M22ppkZzb3sC9u4acUZK5IBr9nsLSByctTOoUI3Y5THEMRk7s"
access_token = "774855303321886720-JxdOM3ieqEVck6l0rYJmSIV3A412anW"
access_token_secret = "AoJgcvbtWmikzHaoOwH2rURrzoRwbBGgoV4oEASumNMzF"

In [26]:
# Words to track
WORDS = ['#coronavirus', '#COVID-19', '#COVID19', '#COVID'] #, '#SocialDistancing', '#pandemic']
raw_tweets_topic = "1_RAW_tweets"

In [41]:
class StreamListener(tweepy.StreamListener):
    # This is a class provided by tweepy to access the Twitter Streaming API.

    def on_connect(self):
        # Called initially to connect to the Streaming API
        print("You are now connected to the streaming API.")

    def on_error(self, status_code):
        # On error - if an error occurs, display the error / status code
        print("Error received in kafka producer " + repr(status_code))
        
        return True 
    
    def preprocess(self, tweetData):
        userID = '-'
        tweetText = '-'
        hashTags = []
        location_full_name = '-'
        location_coutry = '-'
        created_at = '-'

        try:
            userID = tweetData["user"]["id"]
            
            if tweetData["truncated"] == True:
                tweetText = tweetData["extended_tweet"]["full_text"]
                hashTags = [ hTag["text"] for hTag in tweetData["extended_tweet"]["entities"]["hashtags"] ]
            else:
                tweetText = tweetData["text"]
                hashTags = [ hTag["text"] for hTag in tweetData["entities"]["hashtags"] ]

            if hashTags == []:
                return {}

            if "place" in tweetData and tweetData["place"] != None:
                location_full_name = tweetData["place"]["full_name"]
                location_coutry = tweetData["place"]["country"]
                
            created_at = tweetData["created_at"]

            rec = {'userID': userID, 
                   'tweetText': tweetText, 
                   'hashTags': hashTags, 
                   'location_full_name': location_full_name,
                   'location_coutry': location_coutry,
                   'created_at': created_at }

            return json.dumps(rec)

        except Exception as ex:
            print('Exception while parsing')
            print(str(ex))

    def on_data(self, data):
        """ This method is called whenever new data arrives from live stream.
        We asynchronously push this data to kafka queue"""
        try:
            parsed = json.loads(data)

            
            if "user" in parsed and "location" in parsed["user"]:
                if parsed["user"]["location"] != None:
                    processed = self.preprocess(parsed)
                    
                    if isinstance(processed, str):
                        producer.produce(raw_tweets_topic, value=processed.encode('utf-8'))
                        producer.flush()

                        print(parsed["text"])

        except Exception as e:
            print("error! : " + str(e))
            return False #stop stream

        return True

    def on_timeout(self):
        return True

In [ ]:
# Create Auth object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# Set up the listener. The 'wait_on_rate_limit=True' is needed to help with Twitter API rate limiting.
listener = StreamListener(api=tweepy.API(wait_on_rate_limit=True, wait_on_rate_limit_notify=True, timeout=60, retry_delay=5, retry_count=10, retry_errors=set([401, 404, 500, 503])))
stream = tweepy.Stream(auth=auth, listener=listener)
print("Tracking: " + str(WORDS))
stream.filter(track=WORDS, languages = ['en'])

Tracking: ['#coronavirus', '#COVID-19', '#COVID19', '#COVID']
You are now connected to the streaming API.
Join our webinar on Wed., Sept. 1 at 10 a.m. EST as we keep you informed of new #COVID legislative updates regardin… https://t.co/kzYw2InYJQ
India achieves unprecedented milestone!

5️⃣0️⃣% of the eligible population inoculated with the first dose of… https://t.co/rD4cMirIqf
RT @MSF_canada: #COVID19 has provided further opportunity to enforce dehumanizing treatment on #Rohingya, with military authorities denying…
RT @MartinKulldorff: Obesity increases the risk of death from #COVID. Now is the perfect time to exercise more outdoors and to eat healthy.…
RT @disclosetv: JUST IN - BBC presenter Lisa Shaw (44) died due to complications from the AstraZeneca #COVID19 vaccine just over three week…
RT @EricG1247: If you compare mask mandates or anything related to #COVID19 prevention to the Holocaust try reading about the Holocaust.

"…
RT @explode007: ICU NURSE #COVID #VACCINE KILLED ALL A

RT @MartinKulldorff: Obesity increases the risk of death from #COVID. Now is the perfect time to exercise more outdoors and to eat healthy.…
RT @angegarrod: New #COVID19 cases UK 25/8/21 

35,547 and rising again. 

Everything is open, no mask mandates except on London transport…
#YogiAdityanath has ordered strict implementation of night curfew in UP. Despite the decreasing number of cases in… https://t.co/4RiGzC0EDI
RT @ANI: "India achieves unprecedented milestone! 50% of the eligible population inoculated with the first dose of the #COVID19 vaccine," t…
RT @MartinKulldorff: Obesity increases the risk of death from #COVID. Now is the perfect time to exercise more outdoors and to eat healthy.…
According to The @Guardian seven destinations to be added to England’s #COVID travel green list.

🟢 Canada, Denmark… https://t.co/URQer00QIx
Best idea ever! . Delta Airlines will ask unvaccinated employees to pay a $200 health insurance surcharge every mon… https://t.co/8vSyYUEzPr
RT @MartinKulld

RT @CDCgov: CDC recommends an additional dose of #COVID19 vaccines for people who are moderately to severely immunocompromised.   

This fa…
Rising #COVID numbers are worrying. Vaccination must pick up pace to avoid serious outcomes in the next wave. 

Ple… https://t.co/xsqZAHOYXN
RT @RepLoisFrankel: It doesn't have to be this way. Please do your part to stop the spread of #COVID19:

😷 Wear a mask. 
🧼 Wash your hands…
RT @RahulGandhi: Rising #COVID numbers are worrying. Vaccination must pick up pace to avoid serious outcomes in the next wave. 

Please tak…
RT @gagggnnooonn: The NHS is preparing to stab 12 year olds without parental consent 😡

#COVIDIOTS #COVID19 #unvaccinated #NoVaccinePasspor…
RT @disclosetv: JUST IN - BBC presenter Lisa Shaw (44) died due to complications from the AstraZeneca #COVID19 vaccine just over three week…
RT @MsLutzmann: Hi 🇨🇦! Did you know AB has more than 1000 new daily cases of #COVID19 (30% of Canada’s new cases but only 10% of the popula…
RT @DailyExpre

RT @GovAbbott: A new #COVID19 antibody infusion center will open in Nacogdoches tomorrow morning.

This treatment is free and available to…
RT @CAMHnews: Data shows that there's a significant rise in #mentalhealth &amp; addiction issues related to #COVID19. Our Physician-in-Chief Dr…
Going back to school and worried about #covid19 transmission? Here is the work of my doctoral student @natduponsel… https://t.co/Ywej8EIH05
😢Unvaccinated, pregnant nurse in #Alabama dies from #Covid19. https://t.co/RdBC8SzBEM
RT @CPHO_Canada: Today’s #COVID19 #Epidemiology in Canada Daily Update: https://t.co/2586g1O35R https://t.co/VqttnT9zEn
.@DukeRegional Chief Medical Director Dr. Adia Ross says the surge in #COVID19 cases from the delta variant impacts… https://t.co/GERelICB8I
#fact : Antibiotics do not work against viruses, only bacteria. #coronavirus #moronavirus #covidiots #covid19 https://t.co/NkiFv3gqMS
RT @DrTeckKhong: It’s incredulous! 

Mum with child who has inflamed tonsils phones 111 - “hav

RT @roinnslainte: As of midnight, Wednesday 25th August, we are reporting 1,866* confirmed cases of #COVID19. 

As of 8am today, 331 COVID-…
I’ve been trying NOT to watch the news or read articles about #COVID19. &amp; #Delta. However, just read they’re only n… https://t.co/pVzIESGnJw
RT @RealSonOfJewel: @BBC Presenter Lisa Shaw (44) died of complications due to the @AstraZeneca #COVID19 vaccine...

But, but, but...the va…
RT @MartinKulldorff: Obesity increases the risk of death from #COVID. Now is the perfect time to exercise more outdoors and to eat healthy.…
RT @UNICEF_Pakistan: Today #COVAX delivered the 1st batch of 3 million doses of US-donated Pfizer #COVID19 vaccines in Pakistan 🇵🇰 with @UN…
RT @ubctvuganda: STATEMENT | Update on Heroes in Health Awards (HIHA) 2021 Edition and ongoing #COVID19 Vaccination exercise. #UBCUpdates…
RT @MartinKulldorff: Obesity increases the risk of death from #COVID. Now is the perfect time to exercise more outdoors and to eat healthy.…
RT @Kanchan